In [18]:
import threading
import torch
import time

In [19]:
# fine grained locking, hold it only for a short time (a little code)

lock = threading.Lock()   # protects total
total = torch.tensor(0, dtype=torch.int32)

def inc(count):
    global total
    for i in range(count):
        lock.acquire()    
        total += 1
        lock.release()

# inc(100)
t1 = threading.Thread(target=inc, args=[1_000_000])
t2 = threading.Thread(target=inc, args=[1_000_000])

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()
print(end-start)
total

29.52744722366333


tensor(2000000, dtype=torch.int32)

In [17]:
import dis
dis.dis("total += 1")

  1           0 LOAD_NAME                0 (total)
              2 LOAD_CONST               0 (1)
              4 INPLACE_ADD
              6 STORE_NAME               0 (total)
              8 LOAD_CONST               1 (None)
             10 RETURN_VALUE


In [20]:
# coarse-grained locking, hold it only for a short time (a little code)

lock = threading.Lock()   # protects total
total = torch.tensor(0, dtype=torch.int32)

def inc(count):
    global total
    lock.acquire()
    for i in range(count):
        total += 1
    lock.release()

# inc(100)
t1 = threading.Thread(target=inc, args=[1_000_000])
t2 = threading.Thread(target=inc, args=[1_000_000])

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()
print(end-start)
total

10.51757025718689


tensor(2000000, dtype=torch.int32)

In [26]:
bank_accounts = {"x": 25, "y": 100, "z": 200} # in dollars
lock = threading.Lock() # protects bank_accounts

def transfer(src, dst, amount):
    with lock:
        #lock.acquire()
        success = False
        if bank_accounts[src] >= amount:
            bank_accounts[src] -= amount
            bank_accounts[dst] += amount
            success = True
        print("transferred" if success else "denied")
        #lock.release() -- automatic, even if there's an exception

transfer("x", "y", 20)
print(bank_accounts)

transferred
{'x': 5, 'y': 120, 'z': 200}


In [27]:
transfer("x", "y", 20)
print(bank_accounts)

denied
{'x': 5, 'y': 120, 'z': 200}


In [28]:
transfer("w", "x", 50)
print(bank_accounts)

KeyError: 'w'

In [29]:
transfer("z", "x", 50)
print(bank_accounts)

transferred
{'x': 55, 'y': 120, 'z': 150}
